# TaskTree Tutorial

In this tutorial we will walk through the capabilities of task trees in pycram.

First we have to import the necessary functionality from pycram.

In [1]:
from pycram.bullet_world import BulletWorld
from pycram.robot_descriptions import robot_description
import pycram.task
from pycram.enums import Arms
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot
from pycram.designators.object_designator import *
import anytree
import pycram.plan_failures

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


Next we will create a bullet world with a PR2 in a kitchen containing milk and cereal.

In [2]:
world = BulletWorld()
pr2 = Object("pr2", "robot", "pr2.urdf")
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])
cereal = Object("cereal", "cereal", "breakfast_cereal.stl", position=[1.3, 0.7, 0.95])
milk_desig = ObjectDesignatorDescription(names=["milk"])
cereal_desig = ObjectDesignatorDescription(names=["cereal"])
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])

Finally, we create a plan where the robot parks his arms, walks to the kitchen counter and picks the cereal and places it on the table. Then we execute the plan.

In [3]:
@pycram.task.with_tree
def plan():
    with simulated_robot:
        ParkArmsAction.Action(Arms.BOTH).perform()
        MoveTorsoAction([0.3]).resolve().perform()
        pickup_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve()
        pickup_arm = pickup_pose.reachable_arms[0]
        NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()
        PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=["front"]).resolve().perform()
        ParkArmsAction([Arms.BOTH]).resolve().perform()

        place_island = SemanticCostmapLocation("kitchen_island_surface", kitchen_desig.resolve(),
                                           cereal_desig.resolve()).resolve()

        place_stand = CostmapLocation(place_island.pose, reachable_for=robot_desig, reachable_arm=pickup_arm).resolve()

        NavigateAction(target_locations=[place_stand.pose]).resolve().perform()

        PlaceAction(cereal_desig, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

        ParkArmsAction([Arms.BOTH]).resolve().perform()

        ParkArmsAction.Action(Arms.BOTH).perform()

plan()


Now we get the task tree from its module and render it. Rendering can be done with any render method described in the anytree package. We will use ascii rendering here for ease of displaying.

In [4]:
tt = pycram.task.task_tree
print(anytree.RenderTree(tt))

no_operation()
└── plan()
    ├── perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
    ├── perform(self = MoveTorsoAction.Action(position=0.3))
    ├── perform(self = NavigateAction.Action(target_location=[[0.7199999690055847, 0.6800000071525574, 0.0], [0.0, 0.0, 0.017233690238890705, 0.9998514889326064]]))
    ├── perform(self = PickUpAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((1.3, 0.7, 0.95), (0.0, 0.0, 0.0, 1.0)), bullet_world_object=<pycram.bullet_world.Object object at 0x7fd89ef19130>), arm='left', grasp='front'))
    ├── perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
    ├── perform(self = NavigateAction.Action(target_location=[[-1.8875000476837158, 2.3592000007629395, 0.0], [-0.0, 0.0, 0.18653630523785847, -0.982448068260205]]))
    ├── perform(self = PlaceAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((-1.408593773841858, 2.6620311737060547, 1

As we see every task in the plan got recorded correctly. It is noticeable that the tree begins with a NoOperation node. This is done because several, not connected, plans that get executed after each other should still appear in the task tree. Hence, a NoOperation node is the root of any tree. If we re-execute the plan we would see them appear in the same tree even though they are not connected.

In [7]:
world.reset_bullet_world()
plan()
print(anytree.RenderTree(tt))

no_operation()
├── plan()
│   ├── perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
│   ├── perform(self = MoveTorsoAction.Action(position=0.3))
│   ├── perform(self = NavigateAction.Action(target_location=[[0.7199999690055847, 0.6800000071525574, 0.0], [0.0, 0.0, 0.017233690238890705, 0.9998514889326064]]))
│   ├── perform(self = PickUpAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((1.3, 0.7, 0.95), (0.0, 0.0, 0.0, 1.0)), bullet_world_object=<pycram.bullet_world.Object object at 0x7fd89ef19130>), arm='left', grasp='front'))
│   ├── perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
│   ├── perform(self = NavigateAction.Action(target_location=[[-1.8875000476837158, 2.3592000007629395, 0.0], [-0.0, 0.0, 0.18653630523785847, -0.982448068260205]]))
│   ├── perform(self = PlaceAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((-1.408593773841858, 2.6620311737060547, 1

Projecting a plan in a new environment with its own task tree that only exists while the projected plan is running can be done with the ``with`` keyword. When this is done, both the bullet world and task tree are saved and new, freshly reset objects are available. At the end of a with block the old state is restored. The root for such things is then called ``simulation()``.

In [8]:
with pycram.task.SimulatedTaskTree() as stt:
    print(anytree.RenderTree(pycram.task.task_tree))
print(anytree.RenderTree(pycram.task.task_tree))

simulation()
no_operation()
├── plan()
│   ├── perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
│   ├── perform(self = MoveTorsoAction.Action(position=0.3))
│   ├── perform(self = NavigateAction.Action(target_location=[[0.7199999690055847, 0.6800000071525574, 0.0], [0.0, 0.0, 0.017233690238890705, 0.9998514889326064]]))
│   ├── perform(self = PickUpAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((1.3, 0.7, 0.95), (0.0, 0.0, 0.0, 1.0)), bullet_world_object=<pycram.bullet_world.Object object at 0x7fd89ef19130>), arm='left', grasp='front'))
│   ├── perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
│   ├── perform(self = NavigateAction.Action(target_location=[[-1.8875000476837158, 2.3592000007629395, 0.0], [-0.0, 0.0, 0.18653630523785847, -0.982448068260205]]))
│   ├── perform(self = PlaceAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((-1.408593773841858, 2.662031

Task tree can be manipulated with ordinary anytree manipulation. If we for example want to discard the second plan, we would write

In [9]:
tt.root.children = (tt.root.children[0],)
print(anytree.RenderTree(tt, style=anytree.render.AsciiStyle()))

no_operation()
+-- plan()
    |-- perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
    |-- perform(self = MoveTorsoAction.Action(position=0.3))
    |-- perform(self = NavigateAction.Action(target_location=[[0.7199999690055847, 0.6800000071525574, 0.0], [0.0, 0.0, 0.017233690238890705, 0.9998514889326064]]))
    |-- perform(self = PickUpAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((1.3, 0.7, 0.95), (0.0, 0.0, 0.0, 1.0)), bullet_world_object=<pycram.bullet_world.Object object at 0x7fd89ef19130>), arm='left', grasp='front'))
    |-- perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
    |-- perform(self = NavigateAction.Action(target_location=[[-1.8875000476837158, 2.3592000007629395, 0.0], [-0.0, 0.0, 0.18653630523785847, -0.982448068260205]]))
    |-- perform(self = PlaceAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((-1.408593773841858, 2.6620311737060547, 1

We can now re-execute this (modified) plan by executing the leaf in pre-ordering iteration using the anytree functionality. This will not append the re-execution to the task tree.

In [10]:
world.reset_bullet_world()
with simulated_robot:
    [node.code.execute() for node in tt.root.leaves]
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))

no_operation()
+-- plan()
    |-- perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
    |-- perform(self = MoveTorsoAction.Action(position=0.3))
    |-- perform(self = NavigateAction.Action(target_location=[[0.7199999690055847, 0.6800000071525574, 0.0], [0.0, 0.0, 0.017233690238890705, 0.9998514889326064]]))
    |-- perform(self = PickUpAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((1.3, 0.7, 0.95), (0.0, 0.0, 0.0, 1.0)), bullet_world_object=<pycram.bullet_world.Object object at 0x7fd89ef19130>), arm='left', grasp='front'))
    |-- perform(self = ParkArmsAction.Action(arm=<Arms.BOTH: 3>))
    |-- perform(self = NavigateAction.Action(target_location=[[-1.8875000476837158, 2.3592000007629395, 0.0], [-0.0, 0.0, 0.18653630523785847, -0.982448068260205]]))
    |-- perform(self = PlaceAction.Action(object_designator=ObjectDesignatorDescription.Object(name='cereal', type='cereal', pose=((-1.408593773841858, 2.6620311737060547, 1

Nodes in the task tree contain additional information about the status and time of a task.

In [11]:
print(pycram.task.task_tree.children[0])

Code: plan() 
 start_time: 2023-04-28 09:45:15.542254 
 Status: TaskStatus.SUCCEEDED 
 end_time: 2023-04-28 09:45:16.349187 
 


The task tree can also be reset to an empty one by invoking

In [12]:
pycram.task.reset_tree()
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))

no_operation()


If a plan fails using the PlanFailure exception, the plan will not stop. Instead, the error will be logged and saved in the task tree as a failed subtask. First let's create a simple failing plan and execute it.

In [13]:
@pycram.task.with_tree
def failing_plan():
    raise pycram.plan_failures.PlanFailure("Oopsie!")

try:
    failing_plan()
except PlanFailure as e:
    print(e)

Oopsie!


We can now investigate the nodes of the tree, and we will see that the tree indeed contains a failed task.

In [14]:
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))
print(pycram.task.task_tree.children[0])

no_operation()
+-- failing_plan()
Code: failing_plan() 
 start_time: 2023-04-28 09:46:57.441941 
 Status: TaskStatus.FAILED 
 end_time: 2023-04-28 09:46:57.442232 
 


In [15]:
world.exit()